In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import calendar
import random
import seaborn as sns
import numpy as np
import itertools
import warnings
from itertools import groupby
warnings.filterwarnings("ignore")

In [2]:
conn = psycopg2.connect(
host = "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c",
  password = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_password",
  database = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_db",
  port = '5432'
)

In [3]:
# query = """select o.id as order_id,
#             o.created_at::date as order_date,
#             o.customer_id as user_id,
#             (o.total_discounts * 100) / o.total_line_items_price as discount_percent,
#             oli.product_id as product_id,
#             oli.name as product_name,
#             o.total_price as order_amount,
#             CASE
#         WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
#          ELSE 'Delivered'
#      END as order_status,
#             oli.quantity as quantity,
#             p.product_type as product_category,
#             oli.price as sp
#             from orders o
#             inner join order_line_item oli on oli.order_id = o.id
#             inner join products p on oli.product_id=p.id"""

In [4]:
query = """select o.id as order_id,
            o.created_at::date as order_date,
            o.customer_id as user_id,
            o.total_line_items_price as total_price,
            o.total_discounts / o.total_line_items_price as discount_percent,
            oli.product_id as product_id,
            oli.name as product_name,
            o.total_price as order_amount,
            CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
         ELSE 'Delivered'
     END as order_status,
            oli.quantity as quantity,
            p.product_type as product_category,
            oli.price as sp
            from orders o
            inner join order_line_item oli on oli.order_id = o.id
            inner join products p on oli.product_id=p.id"""

In [5]:
data = pd.read_sql(query, conn)

In [6]:
# data[data['order_date']<='2023/07/31']
# data[data['order_date']>='2023/01/01']

In [7]:
data["order_date"] = pd.to_datetime(data["order_date"])
data[data["product_id"] == np.nan]
data = data.dropna()
data['WeekCount']=data['order_date'].dt.strftime('%Y-w%U')
data=data.sort_values('order_date')
WeekList=data['WeekCount'].unique()

In [8]:
data = data[data['order_status']=='Delivered']

In [9]:
WeekList = WeekList.tolist()
output_list = []

for i in range(len(WeekList) - 1):
    output_list.append(WeekList[i])
    current_week = int(WeekList[i].split('-w')[1])
    next_week = int(WeekList[i + 1].split('-w')[1])
    missing_weeks = next_week - current_week - 1
    for j in range(1, missing_weeks + 1):
        missing_week = current_week + j
        output_list.append(f'{WeekList[i][:5]}w{missing_week:02d}')

output_list.append(WeekList[-1])

In [10]:
orderuserlevel=data.groupby(['order_id','user_id']).agg(
          order_date=pd.NamedAgg(column='order_date', aggfunc='min'),

    ).reset_index()
orderuserlevel=orderuserlevel.sort_values('order_date')
orderuserlevel['TrxnRank'] = orderuserlevel.groupby('user_id')['order_date'].rank(method='first')
orderuserlevel


data=pd.merge(data,
         orderuserlevel[['order_id','TrxnRank']],
         on="order_id",
         how ='left')
data['New']=0
data['Repeat']=0
data.loc[data['TrxnRank']==1,'New']=1
data.loc[data['TrxnRank']>1,'Repeat']=1

#AOV Contribution
data['ordervalue']=data['quantity']*data['sp']
data['AOVContribution']=data['ordervalue']/data["order_id"].nunique()
data['DeltaDaysinOrders']=(data.groupby('user_id').order_date.shift() - data.order_date).dt.days.abs()
data['DeltaOrderValues']=(data.groupby('user_id').ordervalue.shift() - data.ordervalue)

In [11]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New', 'Churned (Single Use Only)', 'Recent & Loyals', 'Champions', 'Churned (Repeat Use)']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero', 'High Value & Slow Moving', 'Low Value & Slow Moving', 'New Business Drivers', 'Repeat Business Drivers']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
lst = []

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()

    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['f_score'] = userweeklydata['OrdersCount'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
      if ((df['Frequency']==1) & (df['r_score']>2)):
          return "New"
      elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
          return 'Churned (Single Use Only)'
      elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
          return 'Recent & Loyals'
      elif ((df['Frequency'] > 1) & (df['m_score'] >=3) & (df['r_score'] > 2)):
          return 'Champions'
      else:
          return 'Churned (Repeat Use)'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
    ).reset_index()
    produser=data.groupby(['product_name','user_id']).agg(
        New_user=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    produser_newcount=produser.groupby('product_name').agg(
        New_count=pd.NamedAgg(column='New_user', aggfunc='sum')
    ).reset_index()
    data_repeat=data[data['TrxnRank']==2]
    produser_repeatcount=data_repeat.groupby('user_id').agg(
        Repeat_count=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    k=pd.merge(produser,produser_repeatcount,on='user_id',how='left')
    k.fillna(0)
    k1=k.groupby('product_name').agg(
        Repeat_count=pd.NamedAgg(column='Repeat_count', aggfunc='sum')
    ).reset_index()
    productlevel=pd.merge(product_level,produser_newcount, on='product_name', how='left')
    productlevel=pd.merge(productlevel,k1,on='product_name',how='left')

    productlevel['NewRepeatConversionRatio']=productlevel['Repeat_count']/productlevel['New_count']
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels)).astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero'
        elif df['NewRepeatConversion_score'] >2:
            return 'Repeat Business Drivers'
        elif (df['CP_score'] >2):
            return 'New Business Drivers'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving'
        else:
            return 'Low Value & Slow Moving'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x5 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    # final_stats = final_stats.append(x5, ignore_index=True)
    final_stats = pd.concat([final_stats, x5], ignore_index=True)
    print("done")
    break

  except:
    lst.append(x)


done


In [12]:
data = data[~(data['WeekCount'].isin(lst))]
output_list = [item for item in output_list if item not in lst]

In [13]:
avg_delta_days = data["DeltaDaysinOrders"].mean()
avg_delta_days

37.025330587739106

In [14]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New', 'Churned (Single Use Only)', 'Recent & Loyals', 'Champions', 'Churned (Repeat Use)']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero', 'High Value & Slow Moving', 'Low Value & Slow Moving', 'New Business Drivers', 'Repeat Business Drivers']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
avg_delta_days = data["DeltaDaysinOrders"].mean()

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()

    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['value_score'] = userweeklydata['Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
      if ((df['Frequency']==1) & (df['r_score']>2)):
          return "New"
      elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
          return 'Churned (Single Use Only)'
      elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
          return 'Recent & Loyals'
      elif ((df['Frequency'] > 1) & (df['m_score'] >=2) & (df['Recency'] <= avg_delta_days)):
          return 'Champions'
      else:
          return 'Churned (Repeat Use)'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
    ).reset_index()
    produser=data.groupby(['product_name','user_id']).agg(
        New_user=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    produser_newcount=produser.groupby('product_name').agg(
        New_count=pd.NamedAgg(column='New_user', aggfunc='sum')
    ).reset_index()
    data_repeat=data[data['TrxnRank']==2]
    produser_repeatcount=data_repeat.groupby('user_id').agg(
        Repeat_count=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    k=pd.merge(produser,produser_repeatcount,on='user_id',how='left')
    k.fillna(0)
    k1=k.groupby('product_name').agg(
        Repeat_count=pd.NamedAgg(column='Repeat_count', aggfunc='sum')
    ).reset_index()
    productlevel=pd.merge(product_level,produser_newcount, on='product_name', how='left')
    productlevel=pd.merge(productlevel,k1,on='product_name',how='left')

    productlevel['NewRepeatConversionRatio']=productlevel['Repeat_count']/productlevel['New_count']
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels)).astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero'
        elif df['NewRepeatConversion_score'] >2:
            return 'Repeat Business Drivers'
        elif (df['CP_score'] >2):
            return 'New Business Drivers'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving'
        else:
            return 'Low Value & Slow Moving'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x5 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    # final_stats = final_stats.append(x5, ignore_index=True)
    final_stats = pd.concat([final_stats, x5], ignore_index=True)

  except:
    pass

In [15]:
last_tag_df = userweeklydata[["user_id" , "User_Cohort_Label"]]

In [16]:
def group_consecutive(lst):
    result = []
    current_group = []

    for key, group in groupby(lst):
        key = next(group)
        if current_group and current_group[-1] == key:
            continue

        current_group.append(key)

    result.extend(current_group)
    return result

customer_profile_df = final_stats.groupby("user_id").agg(
    Order_First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
    Order_Last_Date=pd.NamedAgg(column='order_date', aggfunc='max'),
    Total_Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
    Total_Order_Counts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
    User_Journey=pd.NamedAgg(column='User_Cohort_Label', aggfunc=lambda x: group_consecutive(x.tolist())),
    User_First_Tag=pd.NamedAgg(column='User_Cohort_Label', aggfunc=lambda x: group_consecutive(x.tolist())[0]),
    Total_Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
    Mean_Delta_Days_in_Orders=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
    Mean_Delta_Order_Values=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean'),
).reset_index()
globalmaxdateweek=final_stats['order_date'].max()
customer_profile_df["AOV"] = customer_profile_df["Total_Order_Value"]/customer_profile_df["Total_Order_Counts"]
customer_profile_df["ASP"] = customer_profile_df["Total_Order_Value"]/customer_profile_df["Total_Quantity"]
customer_profile_df['Recency'] = (globalmaxdateweek - customer_profile_df['Order_Last_Date']).dt.days.astype(int)
customer_profile_df['Maturity']=np.nan
customer_profile_df.loc[customer_profile_df['Total_Order_Counts']>1,"Maturity"]=(customer_profile_df['Order_Last_Date']-customer_profile_df['Order_First_Date']).dt.days.abs()
customer_profile_df['Mean_Delta_Days_in_Orders'] = np.where(customer_profile_df['Total_Order_Counts'] == 1,np.nan,customer_profile_df['Mean_Delta_Days_in_Orders'])
customer_profile_df['Mean_Delta_Order_Values'] = np.where(customer_profile_df['Total_Order_Counts'] == 1,np.nan,customer_profile_df['Mean_Delta_Order_Values'])

In [17]:
kbc=pd.merge(customer_profile_df,last_tag_df, on ="user_id", how="left")

In [18]:
kbc.rename(columns={"User_Cohort_Label": "User_Final_Tag"}, inplace=True)

In [19]:
customer_prdt = data.groupby("user_id").agg(
    Product_Names_List = pd.NamedAgg(column='product_name', aggfunc=lambda x: list(pd.unique(x))),
    Product_Category_List = pd.NamedAgg(column='product_category', aggfunc=lambda x: list(pd.unique(x))),
    Average_Discount_Percentage = pd.NamedAgg(column='discount_percent', aggfunc="mean"),
    Average_Discount_List = pd.NamedAgg(column='discount_percent', aggfunc=lambda x: list(pd.unique(x))),
    AOV_List = pd.NamedAgg(column='total_price', aggfunc=lambda x: list(pd.unique(x))),
).reset_index()

In [20]:
final_df_new_uplaod = pd.merge(kbc, customer_prdt, on="user_id", how="left")

In [21]:
final_df_median_value = final_df_new_uplaod['Average_Discount_Percentage'].median()
final_df_new_uplaod['Discount_Category'] = pd.cut(final_df_new_uplaod['Average_Discount_Percentage'],
                                                 bins=[float('-inf'), 0, final_df_median_value, float('inf')],
                                                 labels=['No Discount', 'Lower Discount', 'Higher Discount'])

In [22]:
query1 = '''select
    created_at::date as OrderDate,
    id as order_id,
    customer_id as user_id,
    CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
        ELSE 'Delivered'
    END as order_status
    from orders;'''

query2 = '''select
    name as product_name,
    order_id,
    product_id,
    quantity,
    price,
    variant_title as variant
    from order_line_item;'''

query3 = '''select
    id as product_id,
    title as product_title,
    product_type as product_category,
    handle,
    tags
    from products;'''

In [23]:
e = pd.read_sql(query1, conn)
f = pd.read_sql(query2, conn)
g = pd.read_sql(query3, conn)

In [24]:
# product_name, product_id, variant, product_title, product_category, handle, tags
parameter_input = "product_category"

In [25]:
e = e[e['order_status']=='Delivered']
e.drop('order_status',axis=1,inplace=True)

In [26]:
e_sorted = e.sort_values(by=['user_id', 'orderdate', 'order_id'])
# e['Transaction_Rank'] = e.groupby(['user_id'])['orderdate'].rank(method='first')

In [27]:
merged_df = pd.merge(f, e, on='order_id')
merged_df.dropna(inplace = True)

In [28]:
final = pd.merge(merged_df, g, on='product_id')
final.dropna(inplace = True)
final['Transaction_Rank'] = final.groupby(['user_id',parameter_input])['orderdate'].rank(method='first')

In [29]:
final['Total Order Value'] = final['quantity'] * final['price']

In [30]:
def main(final, transactionx, transactiony, parameter):

    final1 = final.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    if parameter == "tags":
        final_tags = final1.copy()
        final_tags['tags'] = final_tags['tags'].str.split(',')
        final_tags = final_tags.explode('tags', ignore_index=True)
        final1 = final_tags
    else:
        pass

    #-------------------------------------------------------------------------------------------------------------------------------

    trans_x = final1[final1['Transaction_Rank']==transactionx]
    trans_y = final1[final1['Transaction_Rank']==transactiony]

    #-------------------------------------------------------------------------------------------------------------------------------

    xuser_count = trans_x.groupby(parameter).agg(
        x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()

    yuser_count = trans_y.groupby(parameter).agg(
        y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()


    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on=parameter, how='left')

    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result = main_grouped_for_title.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Conversion_Rate_x_to_y'] = result['y_trxn_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Cart_Penetration_for_x_transaction'] = result['x_trxn_pool']/(result['x_trxn_pool'].sum())
    result['Cart_Penetration_for_y_transaction'] = result['y_trxn_pool']/(result['y_trxn_pool'].sum())

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result.copy()
    result2 = result.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result1.sort_values(by='Conversion_Rate_x_to_y', ascending=False)
    result2 = result2.sort_values(by='Cart_Penetration_for_x_transaction', ascending=False)

    #-------------------------------------------------------------------------------------------------------------------------------

    result1['Mix']= result1['Conversion_Rate_x_to_y']*100/result1['Conversion_Rate_x_to_y'].sum()
    result1['CumuSum']= result1['Mix'].cumsum()
    result1['Conversion Rate Bin']=""
    result1.loc[result1['CumuSum']<=75,"Conversion Rate Bin"]='HighConversion'
    result1.loc[(result1['CumuSum']>75) & (result1['CumuSum']<=101),"Conversion Rate Bin"]='LowConversion'

    result2['Mix']= result2['Cart_Penetration_for_x_transaction']*100/result2['Cart_Penetration_for_x_transaction'].sum()
    result2['CumuSum']= result2['Mix'].cumsum()
    result2['Cart Penetration Bin']=""
    result2.loc[result2['CumuSum']<=75,"Cart Penetration Bin"]='HighCartPenetration'
    result2.loc[(result2['CumuSum']>75) & (result2['CumuSum']<=101),"Cart Penetration Bin"]='LowCartPenetration'

    #-------------------------------------------------------------------------------------------------------------------------------

    result2 = result2[[parameter,'Cart Penetration Bin']]
    final_merged_on_title_x_to_y = pd.merge(result1, result2, on=parameter)

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y = final_merged_on_title_x_to_y[final_merged_on_title_x_to_y['Conversion Rate Bin'] != 0]

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y['Bin'] = final_merged_on_title_x_to_y['Conversion Rate Bin'] + '_' + final_merged_on_title_x_to_y['Cart Penetration Bin']
    # final_merged_on_title_x_to_y.drop(['Conversion Rate Bin','Cart Penetration Bin'], axis=1, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    merged_title_on_bin = final_merged_on_title_x_to_y.groupby('Bin').agg({
        parameter: 'nunique'
    }).reset_index()

    #-------------------------------------------------------------------------------------------------------------------------------

    x = final_merged_on_title_x_to_y.columns
    y = []
    for i in x:
        if "x_" in i:
            i = i.replace("x_", f"{str(transactionx)}_")
        if "_x" in i:
            i = i.replace("_x", f"_{str(transactiony)}")
        if "y_" in i:
            i = i.replace("y_", f"{str(transactiony)}_")
        if "_y" in i:
            i = i.replace("_y", f"_{str(transactiony)}")
        y.append(i)
    final_merged_on_title_x_to_y.columns = y

    #-------------------------------------------------------------------------------------------------------------------------------

    return final_merged_on_title_x_to_y, final_merged_on_title_x_to_y[[parameter, f"Conversion_Rate_{transactionx}_to_{transactiony}", f"Cart_Penetration_for_{transactionx}_transaction", f"Cart_Penetration_for_{transactiony}_transaction"]]

In [31]:
prdt_page, input = main(final, 1, 2, parameter_input)

In [32]:
def component1(final, transactionx, transactiony, parameter):
    prdt_page, input = main(final, transactionx, transactiony, parameter)
    component1_dump = pd.merge(final, prdt_page, on=parameter_input, how='left')
    acq = f"{transactionx}_trxn_pool"
    repeat = f"{transactiony}_trxn_pool"
    conversion = f"Conversion_Rate_{transactionx}_to_{transactiony}"
    cart_pen_1 = f"Cart_Penetration_for_{transactionx}_transaction"
    cart_pen_2 = f"Cart_Penetration_for_{transactiony}_transaction"
    component1_to_show = component1_dump.groupby("Bin").agg(
        Product_Counts = pd.NamedAgg(column = 'product_name', aggfunc = 'nunique'),
        Quantity = pd.NamedAgg(column = 'quantity', aggfunc = 'sum'),
        Acquired_Pool = pd.NamedAgg(column = acq, aggfunc = 'sum'),
        Repeat_Pool = pd.NamedAgg(column = repeat, aggfunc = 'sum'),
        Total_Order_Value = pd.NamedAgg(column = "Total Order Value", aggfunc = 'sum'),
        rename1 = pd.NamedAgg(column = conversion, aggfunc = 'mean'),
        rename2 = pd.NamedAgg(column = cart_pen_1, aggfunc = 'mean'),
        rename3 = pd.NamedAgg(column = cart_pen_2, aggfunc = 'mean')).reset_index()
    component1_to_show.rename(columns = {"rename1": conversion, "rename2": cart_pen_1, "rename3":cart_pen_2}, inplace = True)
    component1_to_show.set_index("Bin", inplace = True)
    return component1_to_show, component1_dump

In [33]:
component_1_to_show, dump = component1(final, 1, 2, parameter_input)

In [34]:
final_df_new_uplaod["Suggested_Assortment"] = ""
for i in range (len(final_df_new_uplaod)):
    already_purchased = len(final_df_new_uplaod.iloc[i, 15])
    if already_purchased < 10:
        categories = final_df_new_uplaod.iloc[i, 16]
        analyse = dump[dump["product_category"].isin(categories)]
        analyse = analyse.sort_values(by = 'Conversion_Rate_1_to_2', ascending = False)
        num = 10 - already_purchased
        sub_assortment = analyse.head(num)["product_name"].unique()
        assortment = sub_assortment
        final_df_new_uplaod.at[i, "Suggested_Assortment"] = [assortment]
    else:
        final_df_new_uplaod.at[i, "Suggested_Assortment"] = (final_df_new_uplaod.iloc[i, 15])[:10]

In [35]:
final_df_new_uplaod = final_df_new_uplaod[['user_id',
 'Order_First_Date',
 'Order_Last_Date',
 'Total_Order_Value',
 'Total_Order_Counts',
 'User_Journey',
 'User_First_Tag',
 'User_Final_Tag',
 'Total_Quantity',
 'Mean_Delta_Days_in_Orders',
 'Mean_Delta_Order_Values',
 'AOV',
 'ASP',
 'Recency',
 'Maturity',
 'Product_Names_List',
 'Product_Category_List',
 'Average_Discount_Percentage',
 'Average_Discount_List',
 'AOV_List',
 'Discount_Category',
 'Suggested_Assortment']]

In [36]:
final_df_new_uplaod['Order_Count_Without_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(
    lambda discount_list: sum(1 for value in discount_list if isinstance(value, (int, float)) and value == 0.0)
)

final_df_new_uplaod['Order_Count_With_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(
    lambda discount_list: sum(1 for value in discount_list if isinstance(value, (int, float)) and value > 0.0)
)


In [37]:
# final_df_new_uplaod['Order_Count_Without_Discount'] = 0
# final_df_new_uplaod['Order_Count_With_Discount'] = 0
# def count_positive_discounts(discount_list):
#     return len([value for value in discount_list if isinstance(value, (int, float)) and value > 0.0])

# def count_zero_discounts(discount_list):
#     return len([value for value in discount_list if isinstance(value, (int, float)) and value == 0.0])
# for index, row in final_df_new_uplaod.iterrows():
#     discount_list = row['Average_Discount_List']
#     final_df_new_uplaod['Order_Count_With_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(count_positive_discounts)
#     final_df_new_uplaod['Order_Count_Without_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(count_zero_discounts)

In [38]:
final_df_new_uplaod["No_of_SKUs"] = final_df_new_uplaod['Product_Names_List'].apply(len)

In [39]:
final_df_new_uplaod.to_csv("Customer_Profile_for_TCR_new_new.csv", index = False)

In [40]:
def component_1(final_df_new_upload):
    df = pd.DataFrame()

    df['user_id'] = final_df_new_upload["user_id"]
    df['Order_First_Date'] = final_df_new_upload['Order_First_Date']
    df['Order_Last_Date'] = final_df_new_upload['Order_Last_Date']
    df['No_of_Orders'] = final_df_new_upload["Total_Order_Counts"]
    df["Tag"] = final_df_new_upload["User_Final_Tag"]
    df["AOV"] = final_df_new_upload["AOV"]
    df["Recency"] = final_df_new_upload["Recency"]
    df["Discount_Percentage"] = final_df_new_upload["Average_Discount_Percentage"]
    df["No_of_SKUs"] = final_df_new_uplaod['No_of_SKUs']

    return df

result = component_1(final_df_new_uplaod)

In [41]:
result

,user_id,Order_First_Date,Order_Last_Date,No_of_Orders,Tag,AOV,Recency,Discount_Percentage,No_of_SKUs
0,2.912877e+12,2021-07-09,2023-12-01,4,Churned (Repeat Use),576.000000,23,0.285000,3
1,3.554806e+12,2020-10-15,2020-12-02,35,Churned (Repeat Use),739.428571,1117,0.996478,33
2,3.555623e+12,2023-01-11,2023-01-11,1,Churned (Single Use Only),2100.000000,347,0.600000,1
3,3.555652e+12,2022-10-23,2022-10-23,1,Churned (Single Use Only),1000.000000,427,0.275000,2
4,3.555746e+12,2020-10-10,2022-10-28,15,Churned (Repeat Use),2123.866667,422,0.613038,7
...,...,...,...,...,...,...,...,...,...
44436,6.969379e+12,2023-12-24,2023-12-24,1,New,299.000000,0,0.000000,1
44437,6.969399e+12,2023-12-24,2023-12-24,1,New,299.000000,0,0.000000,1
44438,6.969431e+12,2023-12-24,2023-12-24,1,New,1850.000000,0,0.000000,2
44439,6.969458e+12,2023-12-24,2023-12-24,1,New,199.000000,0,0.000000,1


In [42]:
final_df_new_uplaod

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Product_Names_List,Product_Category_List,Average_Discount_Percentage,Average_Discount_List,AOV_List,Discount_Category,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New, Churned (Repeat Use)]",New,Churned (Repeat Use),5,291.666667,...,[Thogarihunkal Estate Coffee - French Press / ...,"[Single Origin Coffee, Coffee]",0.285000,"[0.24, 0.5, 0.4, 0.0]","[585.0, 980.0, 490.0, 249.0]",Higher Discount,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyals, Champions]",Recent & Loyals,Churned (Repeat Use),35,2.257143,...,[Baba Budangiri Estate Coffee - French Press /...,"[Speciality Coffee Series, Small Batch Series,...",0.996478,"[0.9981818181818182, 0.9993746091307066, 0.998...","[550.0, 1599.0, 585.0, 575.0, 600.0, 630.0]",Higher Discount,[Baba Budangiri Estate Coffee - French Press /...,0,8,33
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New],New,Churned (Single Use Only),1,NaN,...,[Brazil Cerrado Coffee (Latin America) - Whole...,[],0.600000,[0.6],[2100.0],Higher Discount,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New],New,Churned (Single Use Only),2,NaN,...,[Mullayangiri Estate Coffee - 250 gm / French ...,"[Single Origin Coffee, Coffee Accessories]",0.275000,[0.275],[1000.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,2
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New, Champions, Churned (Repeat Use), Champio...",New,Churned (Repeat Use),20,53.428571,...,[Thippanahalli Estate Coffee - French Press / ...,"[Single Origin Coffee, Speciality Coffee Serie...",0.613038,"[0.0, 0.9981818181818182, 1.0, 0.2101105845181...","[585.0, 550.0, 440.0, 11394.0, 1050.0, 12999.0]",Higher Discount,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44436,6.969379e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,[South Indian Traditional Filter Coffee - Dark...,[South Indian Filter Coffee],0.000000,[0.0],[299.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1
44437,6.969399e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,[South Indian Traditional Filter Coffee - Dark...,[South Indian Filter Coffee],0.000000,[0.0],[299.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1
44438,6.969431e+12,2023-12-24,2023-12-24,1850.0,1,[New],New,New,2,NaN,...,[Single Malt Whiskey Barrel Aged Coffee - Espr...,"[Small Batch Series, Speciality Coffee Series]",0.000000,[0.0],[1850.0],No Discount,[[Baba Budangiri Estate Coffee - Whole Beans /...,1,0,2
44439,6.969458e+12,2023-12-24,2023-12-24,199.0,1,[New],New,New,1,NaN,...,[South Indian Traditional Filter Coffee - Dark...,[South Indian Filter Coffee],0.000000,[0.0],[199.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1


In [43]:
final_df_median_value_aov = final_df_new_uplaod['AOV'].median()
final_df_new_uplaod['AOV_Category'] = pd.cut(final_df_new_uplaod['AOV'],
                                                 bins=[float('-inf'), final_df_median_value_aov, float('inf')],
                                                 labels=['Lower AOV', 'Higher AOV'])

In [44]:
final_df_median_value_order_count = final_df_new_uplaod['Total_Order_Counts'].median()
final_df_new_uplaod['Order_Count_Category'] = pd.cut(final_df_new_uplaod['Total_Order_Counts'],
                                                 bins=[float('-inf'), final_df_median_value_order_count, float('inf')],
                                                 labels=['Lower Orders', 'Higher Orders'])

In [45]:
final_df_new_uplaod

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Average_Discount_Percentage,Average_Discount_List,AOV_List,Discount_Category,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New, Churned (Repeat Use)]",New,Churned (Repeat Use),5,291.666667,...,0.285000,"[0.24, 0.5, 0.4, 0.0]","[585.0, 980.0, 490.0, 249.0]",Higher Discount,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3,Higher AOV,Higher Orders
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyals, Champions]",Recent & Loyals,Churned (Repeat Use),35,2.257143,...,0.996478,"[0.9981818181818182, 0.9993746091307066, 0.998...","[550.0, 1599.0, 585.0, 575.0, 600.0, 630.0]",Higher Discount,[Baba Budangiri Estate Coffee - French Press /...,0,8,33,Higher AOV,Higher Orders
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New],New,Churned (Single Use Only),1,NaN,...,0.600000,[0.6],[2100.0],Higher Discount,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1,Higher AOV,Lower Orders
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New],New,Churned (Single Use Only),2,NaN,...,0.275000,[0.275],[1000.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,2,Higher AOV,Lower Orders
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New, Champions, Churned (Repeat Use), Champio...",New,Churned (Repeat Use),20,53.428571,...,0.613038,"[0.0, 0.9981818181818182, 1.0, 0.2101105845181...","[585.0, 550.0, 440.0, 11394.0, 1050.0, 12999.0]",Higher Discount,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7,Higher AOV,Higher Orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44436,6.969379e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,0.000000,[0.0],[299.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders
44437,6.969399e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,0.000000,[0.0],[299.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders
44438,6.969431e+12,2023-12-24,2023-12-24,1850.0,1,[New],New,New,2,NaN,...,0.000000,[0.0],[1850.0],No Discount,[[Baba Budangiri Estate Coffee - Whole Beans /...,1,0,2,Higher AOV,Lower Orders
44439,6.969458e+12,2023-12-24,2023-12-24,199.0,1,[New],New,New,1,NaN,...,0.000000,[0.0],[199.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders


In [46]:
def component_2(final_df_new_upload, level):
    df = pd.DataFrame()
    first_part, second_part = level.split(" ")
    if second_part == "AOV":
      filtered_df = final_df_new_uplaod[final_df_new_uplaod["AOV_Category"] == level]
    if second_part == "Orders":
      filtered_df = final_df_new_uplaod[final_df_new_uplaod["Order_Count_Category"] == level]
    if second_part == "Discount":
      filtered_df = final_df_new_uplaod[final_df_new_uplaod["Discount_Category"] == level]


    df['user_id'] = filtered_df["user_id"]
    df['Order_First_Date'] = filtered_df['Order_First_Date']
    df['Order_Last_Date'] = filtered_df['Order_Last_Date']
    df['No_of_Orders'] = filtered_df["Total_Order_Counts"]
    df["Tag"] = filtered_df["User_Final_Tag"]
    df["AOV"] = filtered_df["AOV"]
    df["Recency"] = filtered_df["Recency"]
    df["Discount_Percentage"] = filtered_df["Average_Discount_Percentage"]
    df["No_of_SKUs"] = filtered_df['No_of_SKUs']

    return df

result = component_2(final_df_new_uplaod, "Lower AOV")

In [47]:
result

,user_id,Order_First_Date,Order_Last_Date,No_of_Orders,Tag,AOV,Recency,Discount_Percentage,No_of_SKUs
481,3.568465e+12,2020-09-24,2022-12-25,3,Churned (Repeat Use),543.333333,364,0.648718,3
669,3.570138e+12,2020-11-23,2020-11-23,1,Churned (Single Use Only),550.000000,1126,0.000018,1
677,3.571057e+12,2023-07-05,2023-07-05,1,New,490.000000,172,0.300000,1
738,3.575041e+12,2020-09-30,2020-09-30,1,Churned (Single Use Only),550.000000,1180,0.000000,1
739,3.577734e+12,2020-10-02,2020-10-23,2,Churned (Repeat Use),550.000000,1157,0.000000,2
...,...,...,...,...,...,...,...,...,...
44435,6.969371e+12,2023-12-24,2023-12-24,1,New,299.000000,0,0.000000,1
44436,6.969379e+12,2023-12-24,2023-12-24,1,New,299.000000,0,0.000000,1
44437,6.969399e+12,2023-12-24,2023-12-24,1,New,299.000000,0,0.000000,1
44439,6.969458e+12,2023-12-24,2023-12-24,1,New,199.000000,0,0.000000,1


In [288]:
query_add = """select o.id as order_id,
            oli.product_id as product_id,
            o.customer_id as user_id,
            a.address1 as customer_address,
                        CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
         ELSE 'Delivered'
     END as order_status,
            osa.address1 as delivery_address
            from orders o
            left join addresses a on a.customer_id = o.customer_id
            left join order_shipping_address osa on osa.order_id=o.id
            inner join order_line_item oli on oli.order_id = o.id"""

In [290]:
data1 = pd.read_sql(query_add, conn)

In [292]:
data1["order_status"].unique()

array(['Delivered', 'Cancelled'], dtype=object)

In [293]:
data1 = data1[data1['order_status']=='Delivered']

In [294]:
data1 = data1.dropna(subset=['customer_address'])

In [295]:
filtered_df = data1[data1["customer_address"] != "address not entered"]

In [296]:
filtered_df = filtered_df.drop_duplicates()
filtered_df

,order_id,product_id,user_id,customer_address,order_status,delivery_address
0,5341764845706,5.321602e+12,6.377996e+12,BUNGALOW NO. 71,Delivered,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE..."
2,5341750689930,5.321602e+12,6.704616e+12,"3/4 - 80 - EWS,",Delivered,"3/4 - 80 - EWS,"
4,5341737910410,5.321602e+12,6.801344e+12,Address:3/336-1 404-1 venugopala Swamy Street ...,Delivered,Address:3/336-1 404-1 venugopala Swamy Street ...
8,5341683515530,5.321602e+12,6.909104e+12,Esserpara Jumma Masjid,Delivered,Esserpara Jumma Masjid
13,5341679059082,5.321602e+12,6.837490e+12,BC 04\nchinad Kushalpora,Delivered,BC 04\nchinad Kushalpora
...,...,...,...,...,...,...
169597,5362703335562,5.321602e+12,6.739637e+12,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",Delivered,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1..."
169599,5362445484170,5.321648e+12,6.968182e+12,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",Delivered,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves..."
169600,5361322983562,5.320286e+12,6.955867e+12,"shubasree complex, 4A/18, dharmatala road, fla...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla..."
169601,5361322983562,5.320286e+12,6.955867e+12,"188, Palash Society, Green Dr. Road, Shankar K...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla..."


In [297]:
# import pandas as pd
# import re

# def clean_and_compare_addresses(customer_address, delivery_address):
#     # Remove punctuation and split into individual words
#     customer_words = set(re.findall(r'\b\w+\b', customer_address.lower()))
#     delivery_words = set(re.findall(r'\b\w+\b', delivery_address.lower()))

#     # Check if there are no words in customer_address
#     if not customer_words:
#         return "Gifting"

#     # Calculate the percentage of common words
#     common_words_percentage = len(customer_words.intersection(delivery_words)) / len(customer_words) * 100

#     # Check if at least 70% of words in customer_address are present in delivery_address
#     if common_words_percentage >= 70:
#         result = "Personal Consumption"
#     else:
#         result = "Gifting"

#     return result

# # Apply the function to each row
# filtered_df['result'] = filtered_df.apply(lambda row: clean_and_compare_addresses(row['customer_address'], row['delivery_address']), axis=1)

# # Calculate common_words_percentage for each row
# filtered_df['common_words_percentage'] = filtered_df.apply(lambda row: len(set(re.findall(r'\b\w+\b', row['customer_address'].lower())).intersection(set(re.findall(r'\b\w+\b', row['delivery_address'].lower())))) / len(set(re.findall(r'\b\w+\b', row['customer_address'].lower()))) * 100 if len(set(re.findall(r'\b\w+\b', row['customer_address'].lower()))) > 0 else 0, axis=1)

# # Display the DataFrame with the new column
# print(filtered_df)


In [298]:
filtered_df

,order_id,product_id,user_id,customer_address,order_status,delivery_address
0,5341764845706,5.321602e+12,6.377996e+12,BUNGALOW NO. 71,Delivered,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE..."
2,5341750689930,5.321602e+12,6.704616e+12,"3/4 - 80 - EWS,",Delivered,"3/4 - 80 - EWS,"
4,5341737910410,5.321602e+12,6.801344e+12,Address:3/336-1 404-1 venugopala Swamy Street ...,Delivered,Address:3/336-1 404-1 venugopala Swamy Street ...
8,5341683515530,5.321602e+12,6.909104e+12,Esserpara Jumma Masjid,Delivered,Esserpara Jumma Masjid
13,5341679059082,5.321602e+12,6.837490e+12,BC 04\nchinad Kushalpora,Delivered,BC 04\nchinad Kushalpora
...,...,...,...,...,...,...
169597,5362703335562,5.321602e+12,6.739637e+12,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",Delivered,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1..."
169599,5362445484170,5.321648e+12,6.968182e+12,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",Delivered,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves..."
169600,5361322983562,5.320286e+12,6.955867e+12,"shubasree complex, 4A/18, dharmatala road, fla...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla..."
169601,5361322983562,5.320286e+12,6.955867e+12,"188, Palash Society, Green Dr. Road, Shankar K...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla..."


In [299]:
customer_address = filtered_df.groupby(["user_id"]).agg(
    Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
).reset_index()

In [300]:
customer_address["Delivery_Address_Counts"].iloc[20]

array([['B-1302, Bestech Park View Spa, Sector 47', 1]], dtype=object)

In [301]:
customer_address["user_id"].iloc[0]

2912876953738.0

In [304]:
import pandas as pd

# Assuming you have a DataFrame named customer_address
# with columns 'user_id' and 'Delivery_Address_Counts'

# Create a new DataFrame to store the results
result_rows = []

for i in range(len(customer_address)):
    user_id = customer_address.at[i, 'user_id']
    data = customer_address.at[i, 'Delivery_Address_Counts']

    # Find the index of the maximum count
    max_index = max(range(len(data)), key=lambda x: data[x][1])

    # Add the primary address to the result DataFrame
    result_rows.append({'user_id': user_id,
                        'address_type': 'primary',
                        'delivery_address': data[max_index][0],
                        'count': data[max_index][1]})

    # Add the rest of the addresses as gifting addresses to the result DataFrame
    result_rows.extend({'user_id': user_id,
                        'address_type': 'gifting',
                        'delivery_address': element,
                        'count': count} for j, (element, count) in enumerate(data) if j != max_index)

# Create the result DataFrame from the list of rows
result_df = pd.DataFrame(result_rows, columns=['user_id', 'address_type', 'delivery_address', 'count'])


In [306]:
result_df

,user_id,address_type,delivery_address,count
0,2.912877e+12,primary,LODHA Aurum Grande. Kanjurmarg East,6
1,2.912877e+12,gifting,Electronics City Phase 1 Electronic City,3
2,2.912877e+12,gifting,"269/127 Birhana, Devi dayal Marg , Lucknow",3
3,3.554806e+12,primary,"B109 Subhodaya Laurels,",60
4,3.554806e+12,gifting,#14 Kalpavriksha Apts Race Course Road,60
...,...,...,...,...
49580,6.970257e+12,primary,c-4/11 model town,1
49581,6.970262e+12,primary,do no 74_6/1_2,1
49582,6.970280e+12,primary,27/259 vasanthapeta,1
49583,6.970289e+12,primary,"141B, LIG, Suryanagar Phase 1, KHB Layout, Cha...",1


In [305]:
result_df_tag = result_df[['user_id', 'address_type', 'delivery_address']]
result_df_tag

,user_id,address_type,delivery_address
0,2.912877e+12,primary,LODHA Aurum Grande. Kanjurmarg East
1,2.912877e+12,gifting,Electronics City Phase 1 Electronic City
2,2.912877e+12,gifting,"269/127 Birhana, Devi dayal Marg , Lucknow"
3,3.554806e+12,primary,"B109 Subhodaya Laurels,"
4,3.554806e+12,gifting,#14 Kalpavriksha Apts Race Course Road
...,...,...,...
49580,6.970257e+12,primary,c-4/11 model town
49581,6.970262e+12,primary,do no 74_6/1_2
49582,6.970280e+12,primary,27/259 vasanthapeta
49583,6.970289e+12,primary,"141B, LIG, Suryanagar Phase 1, KHB Layout, Cha..."


In [307]:
final_df_with_tagging = pd.merge(filtered_df, result_df_tag, on=["user_id", "delivery_address"], how="left")

In [308]:
final_df_with_tagging

,order_id,product_id,user_id,customer_address,order_status,delivery_address,address_type
0,5341764845706,5.321602e+12,6.377996e+12,BUNGALOW NO. 71,Delivered,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE...",gifting
1,5341750689930,5.321602e+12,6.704616e+12,"3/4 - 80 - EWS,",Delivered,"3/4 - 80 - EWS,",primary
2,5341737910410,5.321602e+12,6.801344e+12,Address:3/336-1 404-1 venugopala Swamy Street ...,Delivered,Address:3/336-1 404-1 venugopala Swamy Street ...,primary
3,5341683515530,5.321602e+12,6.909104e+12,Esserpara Jumma Masjid,Delivered,Esserpara Jumma Masjid,primary
4,5341679059082,5.321602e+12,6.837490e+12,BC 04\nchinad Kushalpora,Delivered,BC 04\nchinad Kushalpora,primary
...,...,...,...,...,...,...,...
126854,5362703335562,5.321602e+12,6.739637e+12,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",Delivered,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",primary
126855,5362445484170,5.321648e+12,6.968182e+12,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",Delivered,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",primary
126856,5361322983562,5.320286e+12,6.955867e+12,"shubasree complex, 4A/18, dharmatala road, fla...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla...",gifting
126857,5361322983562,5.320286e+12,6.955867e+12,"188, Palash Society, Green Dr. Road, Shankar K...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla...",gifting


In [309]:
final_df_with_tagging.to_csv("final_df_with_tagging.csv", index=False)

In [310]:
# lst = customer_address["Delivery_Address_Counts"].iloc[0][0]
# address = lst[0]
# count = lst[1]

In [311]:
# # Create a new DataFrame to store the extracted information
# new_dataframe = pd.DataFrame(columns=['user_id', 'Address', 'Count'])

# # Iterate through each row of the original DataFrame
# for index, row in customer_address.iterrows():
#     # Extract user_id from the row
#     user_id = row['user_id']

#     # Assuming each row contains a list of lists
#     for lst in row['Delivery_Address_Counts']:
#         # Extract address and count from the nested list
#         address = lst[0]
#         count = lst[1]

#         # Add a new row to the new DataFrame
#         new_dataframe = new_dataframe.append({'user_id': user_id, 'Address': address, 'Count': count}, ignore_index=True)

# # Display the new DataFrame
# print(new_dataframe)

In [312]:
# new_dataframe["user_id"].iloc[0]

In [313]:
# new_dataframe[new_dataframe["user_id"] == 2912876953738.0]

In [314]:
# tag_df = new_dataframe.groupby(["user_id", "customer_address"]).agg(
#     Tag =
# ).reset_index()

In [315]:
# customer_address["customer_address"].iloc[0]

In [316]:
# customer_address["Delivery_Address"].iloc[0]

In [317]:
import pandas as pd

def count_and_quote_unique(x):
    unique_values = pd.unique(x)
    quoted_values = list("'" + unique_values + "'")
    count = len(unique_values)
    return quoted_values, count

customer_address = data1.groupby("user_id").agg(
    Customer_Address=pd.NamedAgg(column='customer_address', aggfunc=lambda x: count_and_quote_unique(x)),
    Delivery_Address=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: count_and_quote_unique(x)),
    Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
).reset_index()

# Extract the counts and quoted values
customer_address['Customer_Address'] = customer_address['Customer_Address'].apply(lambda x: x[0])
customer_address['Delivery_Address'] = customer_address['Delivery_Address'].apply(lambda x: x[0])


In [318]:
customer_address.to_csv("customer_address.csv", index=False)

In [319]:
customer_address["Delivery_Address_Counts"].iloc[0]

array([['LODHA Aurum Grande. Kanjurmarg East', 6],
       ['Electronics City Phase 1 Electronic City', 3],
       ['269/127 Birhana, Devi dayal Marg , Lucknow', 3]], dtype=object)

In [320]:
customer_address["Delivery_Address"].iloc[0]

["'Electronics City Phase 1 Electronic City'",
 "'LODHA Aurum Grande. Kanjurmarg East'",
 "'269/127 Birhana, Devi dayal Marg , Lucknow'"]

In [321]:
final_df_new_uplaod_with_address = pd.merge(final_df_new_uplaod, customer_address, on="user_id", how="left")

In [322]:
final_df_new_uplaod_with_address

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Discount_Category,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category,Customer_Address,Delivery_Address,Delivery_Address_Counts
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New, Churned (Repeat Use)]",New,Churned (Repeat Use),5,291.666667,...,Higher Discount,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3,Higher AOV,Higher Orders,"['269/127 Birhana, Devi dayal Marg , Lucknow',...","['Electronics City Phase 1 Electronic City', '...","[[LODHA Aurum Grande. Kanjurmarg East, 6], [El..."
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyals, Champions]",Recent & Loyals,Churned (Repeat Use),35,2.257143,...,Higher Discount,[Baba Budangiri Estate Coffee - French Press /...,0,8,33,Higher AOV,Higher Orders,"['B109 Subhodaya Laurels', 'D 106, GR Lavender...","['B109 Subhodaya Laurels,', '#14 Kalpavriksha ...","[[B109 Subhodaya Laurels,, 60], [#14 Kalpavrik..."
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New],New,Churned (Single Use Only),1,NaN,...,Higher Discount,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1,Higher AOV,Lower Orders,"['16B-305, Bollineni Hillside']","['16B-305, Bollineni Hillside']","[[16B-305, Bollineni Hillside, 1]]"
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New],New,Churned (Single Use Only),2,NaN,...,Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,2,Higher AOV,Lower Orders,"['3C, Lexus Apartments']","['3C, Lexus Apartments']","[[3C, Lexus Apartments, 2]]"
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New, Champions, Churned (Repeat Use), Champio...",New,Churned (Repeat Use),20,53.428571,...,Higher Discount,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7,Higher AOV,Higher Orders,"['TRIKUTTA TOWERS', 'g wing arkade earth kanju...","['3G2, Maran Enclave, 23 West Vanniar Street,'...",[[g wing arkade earth kanjurmarg east nehru na...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44436,6.969379e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders,['Bekal KUNNIL haddad nagar pallikkera pin 671...,['Bekal KUNNIL haddad nagar pallikkera pin 671...,[[Bekal KUNNIL haddad nagar pallikkera pin 671...
44437,6.969399e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders,"['H.N.928, Mohan Badodiya']","['H.N.928, Mohan Badodiya']","[[H.N.928, Mohan Badodiya, 1]]"
44438,6.969431e+12,2023-12-24,2023-12-24,1850.0,1,[New],New,New,2,NaN,...,No Discount,[[Baba Budangiri Estate Coffee - Whole Beans /...,1,0,2,Higher AOV,Lower Orders,"['1279, Sector 10A']","['1279, Sector 10A']","[[1279, Sector 10A, 2]]"
44439,6.969458e+12,2023-12-24,2023-12-24,199.0,1,[New],New,New,1,NaN,...,No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders,"['5/966, \n12th Street, Ayyappa Nagar, Madipak...","['5/966, \n12th Street, Ayyappa Nagar, Madipak...","[[5/966, \n12th Street, Ayyappa Nagar, Madipak..."


In [323]:
final_df_new_uplaod_with_address.to_csv("fdfnuwa_new.csv", index=False)

In [324]:
final_df_with_tagging

,order_id,product_id,user_id,customer_address,order_status,delivery_address,address_type
0,5341764845706,5.321602e+12,6.377996e+12,BUNGALOW NO. 71,Delivered,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE...",gifting
1,5341750689930,5.321602e+12,6.704616e+12,"3/4 - 80 - EWS,",Delivered,"3/4 - 80 - EWS,",primary
2,5341737910410,5.321602e+12,6.801344e+12,Address:3/336-1 404-1 venugopala Swamy Street ...,Delivered,Address:3/336-1 404-1 venugopala Swamy Street ...,primary
3,5341683515530,5.321602e+12,6.909104e+12,Esserpara Jumma Masjid,Delivered,Esserpara Jumma Masjid,primary
4,5341679059082,5.321602e+12,6.837490e+12,BC 04\nchinad Kushalpora,Delivered,BC 04\nchinad Kushalpora,primary
...,...,...,...,...,...,...,...
126854,5362703335562,5.321602e+12,6.739637e+12,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",Delivered,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",primary
126855,5362445484170,5.321648e+12,6.968182e+12,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",Delivered,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",primary
126856,5361322983562,5.320286e+12,6.955867e+12,"shubasree complex, 4A/18, dharmatala road, fla...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla...",gifting
126857,5361322983562,5.320286e+12,6.955867e+12,"188, Palash Society, Green Dr. Road, Shankar K...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla...",gifting


In [325]:

# Count the number of tags with 'primary' for each user_id
primary_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'primary'].groupby('user_id').size().reset_index(name='primary_count')

# Count the number of tags with 'gifting' for each user_id
gifting_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'gifting'].groupby('user_id').size().reset_index(name='gifting_count')

# Merge the two DataFrames on user_id
result = pd.merge(primary_count, gifting_count, on='user_id', how='outer').fillna(0)



In [326]:
result

,user_id,primary_count,gifting_count
0,2.912877e+12,6,6.0
1,3.554806e+12,60,132.0
2,3.555623e+12,1,0.0
3,3.555652e+12,2,0.0
4,3.555746e+12,81,54.0
...,...,...,...
44466,6.970257e+12,1,0.0
44467,6.970262e+12,1,0.0
44468,6.970280e+12,1,0.0
44469,6.970289e+12,1,0.0


In [327]:
final_df_final_new = pd.merge(final_df_new_uplaod_with_address, result, on="user_id", how="left")

In [328]:
final_df_final_new.to_csv("fdfn.csv", index=False)

In [329]:
final_df_final_new

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category,Customer_Address,Delivery_Address,Delivery_Address_Counts,primary_count,gifting_count
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New, Churned (Repeat Use)]",New,Churned (Repeat Use),5,291.666667,...,1,3,3,Higher AOV,Higher Orders,"['269/127 Birhana, Devi dayal Marg , Lucknow',...","['Electronics City Phase 1 Electronic City', '...","[[LODHA Aurum Grande. Kanjurmarg East, 6], [El...",6.0,6.0
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyals, Champions]",Recent & Loyals,Churned (Repeat Use),35,2.257143,...,0,8,33,Higher AOV,Higher Orders,"['B109 Subhodaya Laurels', 'D 106, GR Lavender...","['B109 Subhodaya Laurels,', '#14 Kalpavriksha ...","[[B109 Subhodaya Laurels,, 60], [#14 Kalpavrik...",60.0,132.0
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New],New,Churned (Single Use Only),1,NaN,...,0,1,1,Higher AOV,Lower Orders,"['16B-305, Bollineni Hillside']","['16B-305, Bollineni Hillside']","[[16B-305, Bollineni Hillside, 1]]",1.0,0.0
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New],New,Churned (Single Use Only),2,NaN,...,0,1,2,Higher AOV,Lower Orders,"['3C, Lexus Apartments']","['3C, Lexus Apartments']","[[3C, Lexus Apartments, 2]]",2.0,0.0
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New, Champions, Churned (Repeat Use), Champio...",New,Churned (Repeat Use),20,53.428571,...,1,3,7,Higher AOV,Higher Orders,"['TRIKUTTA TOWERS', 'g wing arkade earth kanju...","['3G2, Maran Enclave, 23 West Vanniar Street,'...",[[g wing arkade earth kanjurmarg east nehru na...,81.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44436,6.969379e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,1,0,1,Lower AOV,Lower Orders,['Bekal KUNNIL haddad nagar pallikkera pin 671...,['Bekal KUNNIL haddad nagar pallikkera pin 671...,[[Bekal KUNNIL haddad nagar pallikkera pin 671...,1.0,0.0
44437,6.969399e+12,2023-12-24,2023-12-24,299.0,1,[New],New,New,1,NaN,...,1,0,1,Lower AOV,Lower Orders,"['H.N.928, Mohan Badodiya']","['H.N.928, Mohan Badodiya']","[[H.N.928, Mohan Badodiya, 1]]",1.0,0.0
44438,6.969431e+12,2023-12-24,2023-12-24,1850.0,1,[New],New,New,2,NaN,...,1,0,2,Higher AOV,Lower Orders,"['1279, Sector 10A']","['1279, Sector 10A']","[[1279, Sector 10A, 2]]",2.0,0.0
44439,6.969458e+12,2023-12-24,2023-12-24,199.0,1,[New],New,New,1,NaN,...,1,0,1,Lower AOV,Lower Orders,"['5/966, \n12th Street, Ayyappa Nagar, Madipak...","['5/966, \n12th Street, Ayyappa Nagar, Madipak...","[[5/966, \n12th Street, Ayyappa Nagar, Madipak...",1.0,0.0
